# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf
import sys

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard


#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import ann_framework.aux_functions as aux_functions
from ann_framework.aux_functions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(12, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(16, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(12, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc3'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(0.1), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = test_model(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.2)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.2
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(14328, 420)
(14328, 1)
Cross-Validation data (X, y)
(20, 420)
(20, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.57638889 -0.00929121 -0.35818409 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.34722222  0.1032652  -0.25239484 ... -0.45454545  0.25984252
   0.25294702]
 [-0.20833333 -0.10007964 -0.20824656 ... -0.45454545  0.38582677
   0.72049425]
 [-0.20833333 -0.3761614  -0.30528946 ... -0.45454545  0.08661417
   0.29640676]
 [-0.19444444 -0.37297584 -0.12411495 ... -0.45454545  0.05511811
   0.17880983]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Cross-Validation data (X, y)
[[-0.58333333  0.21316698 -0.39525198 ... -0.45454545  0.07086614
   0.16972021]
 [ 0.05555556  0.11866207 -0.1153686  ... -0.27272727 -0.19685039
  -0.12484022]
 [-0.4097

# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(aux_functions.step_decay)

#earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')

shape = (nFeatures*tModel.data_handler.sequence_length)
print(shape)


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

now = datetime.now()
tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)
#,tensorboard = tensorboard
tModel.epochs = 200
tModel.train_model(learningRate_scheduler=lrate, verbose=0,tensorboard = tensorboard)




420
training with cv
Logs: dict_items([('val_loss', 9515.158203125), ('val_mean_squared_error', 9506.583984375), ('loss', 8200.161080740072), ('mean_squared_error', 8192.00446296151), ('lr', 0.001)])
Val_logs: {'loss': 9515.158203125, 'mean_squared_error': 9506.583984375}
Logs: dict_items([('val_loss', 8038.23828125), ('val_mean_squared_error', 8027.6572265625), ('loss', 6992.985370919267), ('mean_squared_error', 6983.561644758079), ('lr', 0.001)])
Val_logs: {'loss': 8038.23828125, 'mean_squared_error': 8027.6572265625}
Logs: dict_items([('val_loss', 6295.9443359375), ('val_mean_squared_error', 6281.2060546875), ('loss', 5546.562948749825), ('mean_squared_error', 5534.138867078448), ('lr', 0.001)])
Val_logs: {'loss': 6295.9443359375, 'mean_squared_error': 6281.2060546875}
Logs: dict_items([('val_loss', 4783.71044921875), ('val_mean_squared_error', 4763.70849609375), ('loss', 4116.413318355013), ('mean_squared_error', 4099.179642107019), ('lr', 0.001)])
Val_logs: {'loss': 4783.710449218

Logs: dict_items([('val_loss', 1180.499755859375), ('val_mean_squared_error', 1115.923583984375), ('loss', 369.2387065024831), ('mean_squared_error', 305.08661827543204), ('lr', 0.001)])
Val_logs: {'loss': 1180.499755859375, 'mean_squared_error': 1115.923583984375}
Logs: dict_items([('val_loss', 1160.5001220703125), ('val_mean_squared_error', 1095.2548828125), ('loss', 365.00012767627743), ('mean_squared_error', 300.08628888665334), ('lr', 0.001)])
Val_logs: {'loss': 1160.5001220703125, 'mean_squared_error': 1095.2548828125}
Logs: dict_items([('val_loss', 1153.59814453125), ('val_mean_squared_error', 1087.81591796875), ('loss', 361.5081746385192), ('mean_squared_error', 295.9541630577202), ('lr', 0.001)])
Val_logs: {'loss': 1153.59814453125, 'mean_squared_error': 1087.81591796875}
Logs: dict_items([('val_loss', 1167.407958984375), ('val_mean_squared_error', 1101.198974609375), ('loss', 358.6544798271141), ('mean_squared_error', 292.66324745403995), ('lr', 0.001)])
Val_logs: {'loss': 11

Logs: dict_items([('val_loss', 1106.0537109375), ('val_mean_squared_error', 1044.707275390625), ('loss', 322.55579252259014), ('mean_squared_error', 261.0672472128983), ('lr', 0.001)])
Val_logs: {'loss': 1106.0537109375, 'mean_squared_error': 1044.707275390625}
Logs: dict_items([('val_loss', 1098.6336669921875), ('val_mean_squared_error', 1037.4935302734375), ('loss', 321.2670318245688), ('mean_squared_error', 260.02222366375605), ('lr', 0.001)])
Val_logs: {'loss': 1098.6336669921875, 'mean_squared_error': 1037.4935302734375}
Logs: dict_items([('val_loss', 1129.325927734375), ('val_mean_squared_error', 1068.4410400390625), ('loss', 320.5033019644664), ('mean_squared_error', 259.505296801935), ('lr', 0.001)])
Val_logs: {'loss': 1129.325927734375, 'mean_squared_error': 1068.4410400390625}
Logs: dict_items([('val_loss', 1089.596435546875), ('val_mean_squared_error', 1028.949951171875), ('loss', 319.86193094514454), ('mean_squared_error', 259.084356004469), ('lr', 0.001)])
Val_logs: {'loss

Logs: dict_items([('val_loss', 1078.3070068359375), ('val_mean_squared_error', 1023.2244873046875), ('loss', 302.90933777911914), ('mean_squared_error', 247.7298785219773), ('lr', 0.001)])
Val_logs: {'loss': 1078.3070068359375, 'mean_squared_error': 1023.2244873046875}
Logs: dict_items([('val_loss', 1115.3629150390625), ('val_mean_squared_error', 1060.450439453125), ('loss', 302.50396972179146), ('mean_squared_error', 247.49398141729972), ('lr', 0.001)])
Val_logs: {'loss': 1115.3629150390625, 'mean_squared_error': 1060.450439453125}
Logs: dict_items([('val_loss', 1090.5233154296875), ('val_mean_squared_error', 1035.787353515625), ('loss', 301.84802907222746), ('mean_squared_error', 247.0149445823955), ('lr', 0.001)])
Val_logs: {'loss': 1090.5233154296875, 'mean_squared_error': 1035.787353515625}
Logs: dict_items([('val_loss', 1097.9219970703125), ('val_mean_squared_error', 1043.327392578125), ('loss', 301.3965597402976), ('mean_squared_error', 246.73092070323406), ('lr', 0.001)])
Val_l

Logs: dict_items([('val_loss', 1089.0338134765625), ('val_mean_squared_error', 1035.24072265625), ('loss', 298.22686665341683), ('mean_squared_error', 244.42199986477527), ('lr', 1e-04)])
Val_logs: {'loss': 1089.0338134765625, 'mean_squared_error': 1035.24072265625}
Logs: dict_items([('val_loss', 1083.6217041015625), ('val_mean_squared_error', 1029.842041015625), ('loss', 298.20817766131), ('mean_squared_error', 244.42238231426774), ('lr', 1e-04)])
Val_logs: {'loss': 1083.6217041015625, 'mean_squared_error': 1029.842041015625}
Logs: dict_items([('val_loss', 1089.6251220703125), ('val_mean_squared_error', 1035.873779296875), ('loss', 298.1065561368038), ('mean_squared_error', 244.3489558455799), ('lr', 1e-04)])
Val_logs: {'loss': 1089.6251220703125, 'mean_squared_error': 1035.873779296875}
Logs: dict_items([('val_loss', 1094.380615234375), ('val_mean_squared_error', 1040.6531982421875), ('loss', 298.039191999494), ('mean_squared_error', 244.2973707785891), ('lr', 1e-04)])
Val_logs: {'lo

Logs: dict_items([('val_loss', 1086.8603515625), ('val_mean_squared_error', 1033.7603759765625), ('loss', 296.16964864810706), ('mean_squared_error', 243.05936798658004), ('lr', 1e-04)])
Val_logs: {'loss': 1086.8603515625, 'mean_squared_error': 1033.7603759765625}
Logs: dict_items([('val_loss', 1091.2012939453125), ('val_mean_squared_error', 1038.1322021484375), ('loss', 296.09678294630703), ('mean_squared_error', 243.00705119632997), ('lr', 1e-04)])
Val_logs: {'loss': 1091.2012939453125, 'mean_squared_error': 1038.1322021484375}
Logs: dict_items([('val_loss', 1091.27001953125), ('val_mean_squared_error', 1038.22802734375), ('loss', 296.02610808627554), ('mean_squared_error', 242.96557269583596), ('lr', 1e-04)])
Val_logs: {'loss': 1091.27001953125, 'mean_squared_error': 1038.22802734375}
Logs: dict_items([('val_loss', 1098.873046875), ('val_mean_squared_error', 1045.86279296875), ('loss', 295.94219149403705), ('mean_squared_error', 242.9107111979303), ('lr', 1e-04)])
Val_logs: {'loss':

Logs: dict_items([('val_loss', 1086.6673583984375), ('val_mean_squared_error', 1034.4459228515625), ('loss', 293.75695027192165), ('mean_squared_error', 241.5131365905455), ('lr', 1e-04)])
Val_logs: {'loss': 1086.6673583984375, 'mean_squared_error': 1034.4459228515625}
Logs: dict_items([('val_loss', 1080.4544677734375), ('val_mean_squared_error', 1028.243408203125), ('loss', 293.6588269036153), ('mean_squared_error', 241.453926482592), ('lr', 1e-04)])
Val_logs: {'loss': 1080.4544677734375, 'mean_squared_error': 1028.243408203125}
Logs: dict_items([('val_loss', 1086.2666015625), ('val_mean_squared_error', 1034.0950927734375), ('loss', 293.60598679909424), ('mean_squared_error', 241.41569944176177), ('lr', 1e-04)])
Val_logs: {'loss': 1086.2666015625, 'mean_squared_error': 1034.0950927734375}
Logs: dict_items([('val_loss', 1082.54443359375), ('val_mean_squared_error', 1030.395263671875), ('loss', 293.4832118559922), ('mean_squared_error', 241.32038860651184), ('lr', 1e-04)])
Val_logs: {'l

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

20/20 [==============================] - 0s 140us/step
scores
Engine 1, Predicted RUL 102.64065551757812, Rounded RUL 102.0, Real RUL 112.0
Engine 2, Predicted RUL 71.6578140258789, Rounded RUL 71.0, Real RUL 98.0
Engine 3, Predicted RUL 86.17501831054688, Rounded RUL 86.0, Real RUL 69.0
Engine 4, Predicted RUL 97.93231964111328, Rounded RUL 97.0, Real RUL 82.0
Engine 5, Predicted RUL -1.5148717164993286, Rounded RUL -2.0, Real RUL 91.0
Engine 6, Predicted RUL 42.41953659057617, Rounded RUL 42.0, Real RUL 93.0
Engine 7, Predicted RUL 18.83766746520996, Rounded RUL 18.0, Real RUL 91.0
Engine 8, Predicted RUL 111.34894561767578, Rounded RUL 111.0, Real RUL 95.0
Engine 9, Predicted RUL 98.85285949707031, Rounded RUL 98.0, Real RUL 111.0
Engine 10, Predicted RUL 70.53758239746094, Rounded RUL 70.0, Real RUL 96.0
Engine 11, Predicted RUL 9.506735801696777, Rounded RUL 9.0, Real RUL 97.0
Engine 12, Predicted RUL 135.8740997314453, Rounded RUL 135.0, Real RUL 124.0
Engine 13, Predicted RUL 12

In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 168us/step
scores
Engine 1, Predicted RUL 115.45829010009766, Rounded RUL 115.0, Real RUL 112.0
Engine 2, Predicted RUL 119.90253448486328, Rounded RUL 119.0, Real RUL 98.0
Engine 3, Predicted RUL 66.14088439941406, Rounded RUL 66.0, Real RUL 69.0
Engine 4, Predicted RUL 86.63671875, Rounded RUL 86.0, Real RUL 82.0
Engine 5, Predicted RUL 99.13146209716797, Rounded RUL 99.0, Real RUL 91.0
Engine 6, Predicted RUL 106.49949645996094, Rounded RUL 106.0, Real RUL 93.0
Engine 7, Predicted RUL 108.7303695678711, Rounded RUL 108.0, Real RUL 91.0
Engine 8, Predicted RUL 89.63396453857422, Rounded RUL 89.0, Real RUL 95.0
Engine 9, Predicted RUL 119.91996765136719, Rounded RUL 119.0, Real RUL 111.0
Engine 10, Predicted RUL 91.80509185791016, Rounded RUL 91.0, Real RUL 96.0
Engine 11, Predicted RUL 93.23152160644531, Rounded RUL 93.0, Real RUL 97.0
Engine 12, Predicted RUL 104.96480560302734, Rounded RUL 104.0, Real RUL 124.0
Engine 13, Predicted RUL 